In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [76]:
class Simple1DConv:
    def __init__(self, f_size, initializer, optimizer, p =1, stride=1, n_channel_in = 1, n_channel_out= 1):
        self.init = initializer
        self.optimizer = optimizer
        self.n_channel_in = n_channel_in
        self.n_channel_out = n_channel_out
        self.stride = stride
        self.pad = p
        self.n_out = None
        self.f_size = f_size
    
    def forward(self, X):
        self.n_in = X.reshape(-1, 1).shape[0] # number of features
        self.n_samples = X.reshape(-1, 1).shape[1] # number of samples

        self.x_new = X.reshape(self.n_channel_in, self.n_samples, self.n_in) # reshape into 3D array of (layers, number of samples, number of features)
        self.x_new = np.pad(self.x_new, ((0, 0), (0, 0), (self.pad, self.pad))) # Adds padding around row and column for each layer
        
        self.n_out = num_out(self.n_in, self.pad, self.f_size, self.stride) # calculates the number of output from this convolutional layer
        
        output = np.zeros((self.n_channel_in, self.n_samples, self.n_out)) # features

        # initialization of weights and biases
        self.w = self.init.W((self.n_channel_in, self.n_samples, self.f_size))
        self.b = self.init.B((self.n_channel_in, self.n_samples, self.f_size))

        # Loops through each layer, convolutes the rows and columns for each, and performs forward propagation
        for i in range(self.n_channel_in):
            a = np.array([]) # features extracted by filter
            for j in range(0, self.x_new.shape[-1], self.stride):
                if j + self.f_size > self.x_new.shape[-1]:
                    break
                a = np.append(a, np.sum(self.x_new[i][:, j: j + self.f_size][0] @ self.w[i][0] + self.b[i][0]))
     
            output[i] = a
            #a = np.array(a).reshape(1, 1, a.shape[-1]) # reshape into (1, 1, number of features)
            #output = np.append(output, a, axis=0) # appended extracted features to layer

        return output.sum(axis=0)

    
    def backward(self, dA):
        self.db = np.sum(dA, axis=(0, -1))
        n_in = self.x_new.shape[-1] # number of input features
        self.dw = np.zeros((self.n_channel_in, self.n_samples, self.f_size))
        l = dA.shape[-1]
        for i in range(self.n_channel_in):
            layer = self.x_new[i]
            w = np.array([])
            for j in range(0, n_in, self.stride):
                if j + l > n_in:
                    break           
                w = np.append(w, dA[i] @ layer[:, j: j + l][0])        
            self.dw[i] = w

        self.dx = np.zeros((self.n_channel_in, 1, self.f_size))

        a = np.pad(dA, ((0, 0), (0, 0), (self.stride, self.stride)))
        dx = np.ones((self.n_channel_in, dA.shape[1], self.n_out))
        for  i in range(self.n_channel_in):
            layer = a[i]
            weight = self.w[i]
            w = np.array([])
            for j in range(0, a.shape[-1], self.stride):
                if j + self.f_size > a.shape[-1]:
                    break
                w = np.append(w, layer[:, j: j+self.f_size][0] @ weight[0])
            dx[i] = w
        
        self.optimizer(self)
        return dx
            



                


        




def num_out(n_in, pad, f, s):
    """
    ##### self.n_in: number of features
    ##### pad: Number of padding on one side
    ##### s: stride value

    ##### Returns: Number of output of convolution
    """
    n_out = ((n_in + 2*pad - f)/ s) +1
    return int(n_out)





In [5]:
class FC:
    """
    Number of nodes Fully connected layer from n_nodes1 to n_nodes2
    Parameters
    ----------
    n_nodes1 : int
      Number of nodes in the previous layer
    n_nodes2 : int
      Number of nodes in the later layer
    initializer: instance of initialization method
    optimizer: instance of optimization method
    """
    
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.weights = initializer.W(n_nodes1, n_nodes2)
        self.bias = initializer.B(n_nodes2)
        

        
    
    def forward(self, X):
        """
        forward
        Parameters
        ----------
        X : The following forms of ndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : The following forms of ndarray, shape (batch_size, n_nodes2)
            output
        """        
        self.Z = X
        A = X @ self.weights + self.bias
        
        return A
    
    def backward(self, dA):
        """
        Backward
        Parameters
        ----------
        dA : The following forms of ndarray, shape (batch_size, n_nodes2)
            Gradient flowing from behind
        Returns
        ----------
        dZ : The following forms of ndarray, shape (batch_size, n_nodes1)
            Gradient to flow forward
        """
        
        # update
        self.dB = np.sum(dA, axis=0)
        self.dW = self.Z.T @ dA 
        self.dZ = dA @ self.weights.T
        
        self.optimizer.update(self)
        
        return self.dZ


In [6]:
class SimpleInitializer:
    """
    Simple initialization with Gaussian distribution
    Parameters
    ----------
    sigma : float
      Standard deviation of Gaussian distribution
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, shape):
        """
        Weight initialization
        Parameters
        ----------
        Shape: tuple
        Returns
        ----------
        W :
        """
        w = self.sigma * np.random.randn(*shape)
        
        return w
    
    def B(self, shape):
        """
        Bias initialization
        Parameters
        ----------
        shape : tuple
          Number of nodes in the later layer

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(*shape)
        return B

In [7]:
class Relu:
    def __init__(self) -> None:
        pass

    def forward(self, X):
        self.A = X
        return np.clip(X, 0, None)
    
    def backward(self, X):
        a = X > 0
        return X * np.clip(np.sign(self.A), 0, None)


class Softmax():
        def __init__(self) -> None:
            pass


        def forward(self, a):
            numerator = np.exp(a)
            self.Z = numerator / np.sum(np.exp(a), axis=1).reshape(-1, 1)
            return self.dZ
        
        def backward(self, Y):
             self.loss = self.loss_func(Y)

             return self.dZ - Y
        
        def loss_func(self, Y, Z = None):             
            if Z == None:
                Z = self.Z
            return (-1) * np.average(np.sum(Y * np.log(Z)), axis=1)

            
            
        
    

In [8]:
class AdaGrab:
    def __init__(self, lr):
        self.lr = lr 
        self.ssg = 0

    def update(self, layer):
        
        self.ssg += np.sum(layer.weights ** 2)  
        adaptive_lr = self.lr / np.sqrt(self.ssg + 1e-8)
        layer.weights -= (self.lr - adaptive_lr) * (layer.weights)
        layer.bias -= (self.lr - adaptive_lr) * layer.bias

        
    

In [77]:
model = Simple1DConv(3, SimpleInitializer(0.05), AdaGrab(0.01))

input = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

model.forward(input)


input = input.reshape((1, 1, input.shape[-1]))

model.backward(input)

array([[[0.03419906, 0.06430012, 0.09440119, 0.12450226, 0.15460332,
         0.18470439, 0.21480545, 0.24490652, 0.12432047]]])

In [58]:
input = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
input = np.pad(input, (1, 1))
print(input[0:8])
print(input[1:9])
print(input[2:10])

[0 1 2 3 4 5 6 7]
[1 2 3 4 5 6 7 8]
[2 3 4 5 6 7 8 9]


In [11]:
class GetMiniBatch:
    """
Iterator to get a mini-batch

    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random seed
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int_)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]